In [1]:
!pip install xai-sdk

   ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.5 MB ? eta -:--:--
   --------- ------------------------------ 1.0/4.5 MB 2.3 MB/s eta 0:00:02
   ---------------- ----------------------- 1.8/4.5 MB 3.6 MB/s eta 0:00:01
   ---------------- ----------------------- 1.8/4.5 MB 3.6 MB/s eta 0:00:01
   --------------------- ------------------ 2.4/4.5 MB 2.7 MB/s eta 0:00:01
   ------------------------- -------------- 2.9/4.5 MB 2.3 MB/s eta 0:00:01
   ------------------------------------- -- 4.2/4.5 MB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 4.5/4.5 MB 2.8 MB/s eta 0:00:00

  Attempting uninstall: packaging

    Found existing installation: packaging 24.2

    Uninstalling packaging-24.2:

      Successfully uninstalled packaging-24.2

  Attempting uninstall: grpcio

    Found existing installation: grpcio 1.70.0

    Uninstallin

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 25.1.1 requires conda-libmamba-solver>=24.11.0, but you have conda-libmamba-solver 24.1.0 which is incompatible.
langchainhub 0.1.21 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.
langchain-core 0.3.39 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.
langchain-openai 0.1.25 requires langchain-core<0.3.0,>=0.2.40, but you have langchain-core 0.3.39 which is incompatible.
mlflow-skinny 2.17.0 requires packaging<25, but you have packaging 25.0 which is incompatible.
prefect 3.0.2 requires click<8.2,>=8.0, but you have click 8.2.1 which is incompatible.
prefect 3.0.2 requires packaging<24.3,>=21.3, but you have packaging 25.0 which is incompatible.
prefect 3.0.2 requires rich<14.0,>=11.0, but you have rich 14.0.0 which is incompatible.
prefect 3.0

In [2]:
import os

from xai_sdk import Client
from xai_sdk.chat import system, user

# Initialize the client with extended timeout
client = Client(
    api_key=os.getenv("XAI_API_KEY"),
    timeout=3600,  # Extended timeout for reasoning-intensive tasks
)


In [3]:
# Create a new chat with system prompt tailored to the prescription assistant
chat = client.chat.create(
    model="grok-4",
    messages=[
        system("""
        You are MedGuide AI, a helpful and intelligent assistant that helps users understand their medical prescriptions.
        You explain each medicine's price, availability, and prescribed duration in a clear and concise manner.
        """),
    ],
)

In [ ]:
# Append a user prompt simulating a text-only prescription input
chat.append(
    user("""
My prescription says:
- Paracetamol 500mg, take twice a day for 5 days.
- Azithromycin 250mg, once a day for 3 days.

Can you explain the duration and check if these medicines are commonly available?
""")
)

# Get the response from Grok
response = chat.sample()

# Print detailed response information
print("Reasoning Content:")
print(response.reasoning_content)

print("\nFinal Response:")
print(response.content)

print("\nNumber of completion tokens:")
print(response.usage.completion_tokens)

print("Number of reasoning tokens:")
print(response.usage.reasoning_tokens)


Reasoning Content:


Final Response:
Below, I'll explain your prescription based on the details you provided. As MedGuide AI, I'll cover the prescribed duration for each medicine, their common availability (based on general knowledge in most countries like the US, UK, or India—availability can vary by location and regulations), and approximate prices (these are estimates and can fluctuate based on brand, location, pharmacy, and whether it's generic or branded; I recommend checking with a local pharmacy for exact details). Remember, I'm not a substitute for professional medical advice—always follow your doctor's instructions and consult them if you have questions.

### 1. **Paracetamol 500mg**
   - **Prescribed Duration**: Take one tablet twice a day (e.g., morning and evening) for 5 days. This means a total course of 10 tablets (2 per day × 5 days). It's commonly used for pain relief, fever, or mild inflammation.
   - **Availability**: Very commonly available over-the-counter (OTC) in 

In [4]:
import base64
from xai_sdk.chat import image

# Set the path to your prescription image
image_path = "data/image1.jpg"

# Start a new chat with Grok 4
chat = client.chat.create(model="grok-4")

# Define the AI's role and behavior
chat.append(
    system(
        "You are MedGuide AI, a helpful and intelligent assistant that reads medical prescriptions and explains each medicine's price, availability, and prescribed duration. Be concise, clear, and accurate."
    )
)


# Function to encode image in base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
        return encoded_string


# Encode the image
base64_image = encode_image(image_path)

# Upload the image with a user query
chat.append(
    user(
        "Please analyze this prescription and tell me the details, price, availability, and duration for each medicine.",
        image(image_url=f"data:image/jpeg;base64,{base64_image}", detail="high"),
    )
)

# Get the AI response
response = chat.sample()
print(response.content)


Below is a clear and concise analysis of the prescription dated 13/10/2022 for Ms. Fiza Mozawala (19/F). I've interpreted the handwritten items based on standard medical terminology (e.g., "1-0-1" means 1 dose in the morning, 0 in the afternoon, and 1 in the evening). For each item, I've provided:

- **Details**: What it is, common use, and dosage.
- **Prescribed Duration**: As noted.
- **Price**: Approximate retail price in INR (based on common Indian pharmacy rates as of 2023; prices may vary by location, brand, and discounts—check current rates at pharmacies like Apollo or 1mg).
- **Availability**: General status in India (prescription or over-the-counter; widely available unless noted).

This prescription appears to be for dental/oral health issues (e.g., infection, pain, gum care). Consult your doctor or pharmacist for exact usage, side effects, or alternatives. The last item is a recommended product, not a medicine.

### 1. Cap. Phexin 500mg
- **Details**: Cephalexin (antibiotic)

In [5]:
print(response.usage)

completion_tokens: 893
prompt_tokens: 1860
total_tokens: 3351
prompt_text_tokens: 68
prompt_image_tokens: 1792
reasoning_tokens: 598
cached_prompt_text_tokens: 4



In [ ]:
import os
import json
import base64
from IPython.display import Markdown
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Dict, List
from xai_sdk import Client
from xai_sdk.chat import system, user, image, tool, tool_result
from firecrawl import FirecrawlApp, ScrapeOptions


# Initialize clients with connection pooling
client = Client(
    api_key=os.getenv("XAI_API_KEY"),
    timeout=3600,
)
fc = FirecrawlApp(api_key=os.getenv("FIRECRAWL_API_KEY"))


def get_medicine_info_fast(name: str) -> Dict:
    """Optimized medicine info fetcher with error handling"""
    try:
        results = fc.search(
            query=f"{name} medicine price availability",  # Shorter query for faster search
            limit=1,
            scrape_options=ScrapeOptions(formats=["markdown"]),
        )
        snippet = results.data[0] if results.data else {}
        return {
            "name": name,
            "info_markdown": snippet.get("markdown", "N/A"),
            "url": snippet.get("url", "N/A"),
            "description": snippet.get("description", "N/A"),
            "status": "success",
        }
    except Exception as e:
        return {
            "name": name,
            "info_markdown": "Error fetching data",
            "url": "N/A",
            "description": f"Error: {str(e)}",
            "status": "error",
        }


def get_multiple_medicines_concurrent(
    medicine_names: List[str], max_workers: int = 5
) -> List[Dict]:
    """Fetch multiple medicine info concurrently"""
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_medicine = {
            executor.submit(get_medicine_info_fast, name): name
            for name in medicine_names
        }

        # Collect results as they complete
        for future in as_completed(future_to_medicine):
            try:
                result = future.result(timeout=30)  # 30 second timeout per request
                results.append(result)
            except Exception as e:
                medicine_name = future_to_medicine[future]
                results.append(
                    {
                        "name": medicine_name,
                        "info_markdown": "Timeout or error",
                        "url": "N/A",
                        "description": f"Error: {str(e)}",
                        "status": "error",
                    }
                )

    return results


# Tool definitions
tool_definitions = [
    tool(
        name="get_medicine_info_fast",
        description="Fetch markdown info, URL, and description for a medicine via Firecrawl (optimized)",
        parameters={
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "Name of the medicine"},
            },
            "required": ["name"],
        },
    ),
    tool(
        name="get_multiple_medicines_concurrent",
        description="Fetch info for multiple medicines concurrently",
        parameters={
            "type": "object",
            "properties": {
                "medicine_names": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of medicine names",
                },
                "max_workers": {
                    "type": "integer",
                    "description": "Maximum concurrent workers (default: 5)",
                    "default": 5,
                },
            },
            "required": ["medicine_names"],
        },
    ),
]

# Map tools to functions
tools_map = {
    "get_medicine_info_fast": get_medicine_info_fast,
    "get_multiple_medicines_concurrent": get_multiple_medicines_concurrent,
}


def encode_image(path: str) -> str:
    """Utility to encode image to base64"""
    try:
        with open(path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    except FileNotFoundError:
        raise FileNotFoundError(f"Image file not found: {path}")
    except Exception as e:
        raise Exception(f"Error encoding image: {str(e)}")


def analyze_prescription_fast(image_path: str) -> str:
    """Main function to analyze prescription with optimizations"""
    try:
        # Encode the prescription image
        encoded_img = encode_image(image_path)

        # Create chat session
        chat = client.chat.create(
            model="grok-4",
            tools=tool_definitions,
            tool_choice="auto",
        )

        # Enhanced system prompt for better extraction
        chat.append(
            system(
                "You are MedGuide AI. Extract ALL medicine names from the prescription image. "
                "If you find multiple medicines, use get_multiple_medicines_concurrent to fetch "
                "all information at once for faster processing. For single medicine, use get_medicine_info_fast. "
                "Create a comprehensive markdown report."
            )
        )

        # User provides the prescription image
        chat.append(
            user(
                "Extract all medicine names from this prescription and get their details efficiently.",
                image(image_url=f"data:image/jpeg;base64,{encoded_img}", detail="high"),
            )
        )

        # Initial model call
        response = chat.sample()
        chat.append(response)

        # Execute tool calls if any
        if response.tool_calls:
            for tc in response.tool_calls:
                func_name = tc.function.name
                func_args = json.loads(tc.function.arguments)

                print(f"Executing {func_name} with args: {func_args}")

                result = tools_map[func_name](**func_args)
                chat.append(tool_result(json.dumps(result)))

            # Request final formatted report
            chat.append(
                user(
                    "Create a comprehensive markdown report with H2 heading for each medicine that contains: Description, "
                    "Typical Duration, Price Information, and Purchase Link"
                )
            )

            # Generate final report
            final = chat.sample()
            return final.content
        else:
            return response.content

    except Exception as e:
        return f"Error analyzing prescription: {str(e)}"



# Run the optimized analysis
image_path = "data/image1.jpg"
result = analyze_prescription_fast(image_path)


Markdown(result)


2025-07-25 18:21:36,163 [DEBUG] Initialized FirecrawlApp with API URL: https://api.firecrawl.dev
2025-07-25 18:21:57,183 [DEBUG] Starting new HTTPS connection (1): api.firecrawl.dev:443
2025-07-25 18:21:57,186 [DEBUG] Starting new HTTPS connection (1): api.firecrawl.dev:443
2025-07-25 18:21:57,187 [DEBUG] Starting new HTTPS connection (1): api.firecrawl.dev:443
2025-07-25 18:21:57,191 [DEBUG] Starting new HTTPS connection (1): api.firecrawl.dev:443


Executing get_multiple_medicines_concurrent with args: {'medicine_names': ['Phexin 500mg', 'Zerodol PT', 'Stolin gum paint', 'Colgate Plax mouth wash']}


2025-07-25 18:21:59,087 [DEBUG] https://api.firecrawl.dev:443 "POST /v1/search HTTP/1.1" 200 12638
2025-07-25 18:21:59,406 [DEBUG] https://api.firecrawl.dev:443 "POST /v1/search HTTP/1.1" 200 58141
2025-07-25 18:22:00,976 [DEBUG] https://api.firecrawl.dev:443 "POST /v1/search HTTP/1.1" 200 20112
2025-07-25 18:22:04,148 [DEBUG] https://api.firecrawl.dev:443 "POST /v1/search HTTP/1.1" 200 129600


In [53]:
Markdown(result)


# Comprehensive Medicine Report

## Phexin 500mg

### Description
Phexin 500mg Capsule contains Cefalexin, an antibiotic used to treat bacterial infections such as urinary tract infections, tonsillitis/pharyngitis, otitis media, bronchitis, cystitis, pyelonephritis, and osteomyelitis. It works by preventing bacteria from forming their protective covering, thus killing them. It is also used to prevent infections after surgery. This is a prescription-based drug and should be taken as directed by a doctor.

### Typical Duration
As per the prescription: 1-0-1 (one in the morning, none in the afternoon, one in the evening) for 5 days.

### Price Information
Available as low as $30.50 for 30 capsules ($1.02 per capsule). Bulk options: $59.80 for 60 capsules ($1.00 per capsule), $88.80 for 90 capsules ($0.99 per capsule).

### Purchase Link
[Buy Phexin 500mg on CheapMedicineShop](https://www.cheapmedicineshop.com/phexin-500-mg-capsule.html)

## Zerodol PT

### Description
Zerodol PT Tablet is a combination medicine containing Aceclofenac (100mg), Paracetamol (325mg), and Tramadol (37.5mg). It is used for pain relief in conditions like rheumatoid arthritis, ankylosing spondylitis, osteoarthritis, muscle pain, back pain, toothache, ear/throat pain, and fever. It works by blocking pain signals and reducing inflammation. This is a prescription-required medication and should not be taken longer than advised.

### Typical Duration
As per the prescription: 1-0-1 (one in the morning, none in the afternoon, one in the evening) for 5 days.

### Price Information
₹16.5 per tablet (strip of 10 tablets). Substitutes available at lower prices, e.g., Hifenac TA at ₹15.1 per tablet (8% cheaper).

### Purchase Link
[Buy Zerodol PT on 1mg](https://www.1mg.com/drugs/zerodol-pt-tablet-160347?srsltid=AfmBOoovnJvMf9R35T_Bzinn_VK1dBXKh0esUmawSEYrcL8AGhnhdmV4)

## Stolin gum paint

### Description
Stolin Gum Astringent is used to treat gum diseases such as bleeding, swollen, spongy, or receding/loose gums. It prevents mouth ulcers and is applied as a gum paint. Shake well before use. This product is suitable for oral care and helps in managing gum-related issues.

### Typical Duration
As per the prescription: 1-0-1 (one in the morning, none in the afternoon, one in the evening) for 3 days.

### Price Information
Pack of 5 x 15ml for $23.74 (or best offer, originally $24.99, 5% off). Other packs: 2x for $13.01 to $15.65.

### Purchase Link
[Buy Stolin Gum Paint on eBay](https://www.ebay.com/itm/266704508636)

## Colgate Plax mouth wash

### Description
Colgate Plax Complete Care Mouthwash is an alcohol-free mouthwash that provides fresh breath, fights bad breath germs, and supports oral hygiene. It has a peppermint flavor and is gentle on the mouth without burning sensation. Suitable for daily use to maintain oral health.

### Typical Duration
As per the prescription: For 1 week.

### Price Information
250ml/8.5oz bottle for $10.76 ($1.27 per fluid ounce, 27% off typical price of $14.69).

### Purchase Link
[Buy Colgate Plax Mouthwash on Amazon](https://www.amazon.com/Colgate-Plax-Complete-Care-Mouthwash/dp/B00I6F7612)